In [6]:
import pandas as pd
import numpy as np
import warnings

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, roc_auc_score
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn import ensemble
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

In [41]:
## 读取数据
train_data = pd.read_csv('./data/train.csv',sep=',')
test_data  = pd.read_csv('./data/test.csv',sep=',')

In [42]:
train_data

,id,asm_commands_add,asm_commands_call,asm_commands_cdq,asm_commands_cld,asm_commands_cli,asm_commands_cmc,asm_commands_cmp,asm_commands_cwd,asm_commands_daa,...,asm_commands_sti,asm_commands_stos,asm_commands_sub,asm_commands_test,asm_commands_wait,asm_commands_xchg,asm_commands_xor,line_count_asm,size_asm,label
0,0,459,236,1,1,0,0,183,0,13,...,2,2,841,6,0,5,15,7937,460288,7
1,1,696,1310,0,0,36,0,263,0,249,...,0,2,2755,281,0,9,252,37552,2177970,5
2,2,554,1423,0,0,61,0,763,0,85,...,20,0,2736,151,21,0,304,15245,884176,8
3,3,14,164,0,0,0,0,1,0,0,...,1,0,303,6,3,0,2,91393,5300736,6
4,4,931,3728,4,0,0,0,1641,0,40,...,0,197,8172,1429,42,0,970,9335,541421,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,9,162,0,0,0,0,1,0,0,...,4,0,145,16,0,0,2,77825,4513792,6
49996,49996,561,234,0,0,6,0,122,0,171,...,0,0,620,215,0,1,215,83457,4840448,8
49997,49997,168,134,7,7,2,3,48,3,51,...,4,8,189,35,6,28,66,12767,740447,5
49998,49998,103,80,0,3,5,0,34,0,42,...,0,7,292,12,6,13,22,11009,638464,3


In [131]:
feature_cols = [col for col in train_data.columns if col not in ['id','label']]
x_train = train_data[feature_cols]
y_train = train_data['label']
x_test  = test_data[feature_cols]

In [92]:
from sklearn.metrics import accuracy_score

In [191]:
def cv_model(clf, train_x, train_y, test_x, clf_name='lgb'):
    folds = 10
    seed = 111
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros((train_x.shape[0],9))
    test = np.zeros((test_x.shape[0],9))

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        lgb_model = clf.LGBMClassifier(
            boosting_type="gbdt", num_leaves=31, max_depth=-1, n_estimators=3000, objective='multiclass',
            num_class=9, subsample=0.5, colsample_bytree=0.5, subsample_freq=1, min_child_samples=1,
            learning_rate=0.01, random_state=2021, n_jobs=-1, metric="multi_logloss", importance_type='gain',
            device='gpu',gpu_platform_id=0,gpu_device_id=0
        ) 
        eval_set = [(val_x, val_y)]
        lgb_model.fit(trn_x, trn_y, eval_set=eval_set, verbose=100, early_stopping_rounds=200)
        val_pred = lgb_model.predict_proba(val_x)
        test_pred = lgb_model.predict_proba(test_x)
        
        train[valid_index] = val_pred
        test += test_pred / kf.n_splits
        val_predcol = val_pred.argmax(axis=1)
        print(val_predcol)
        cv_scores.append(accuracy_score(val_y, val_predcol))
        print(cv_scores)
    
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train, test

In [192]:
lgb_train, lgb_test = cv_model_lgb(lgb, x_train, y_train, x_test)

************************************ 1 ************************************
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_logloss: 0.247909
[200]	valid_0's multi_logloss: 0.0475889
[300]	valid_0's multi_logloss: 0.0118215
[400]	valid_0's multi_logloss: 0.00483846
[500]	valid_0's multi_logloss: 0.00308693
[600]	valid_0's multi_logloss: 0.00247629
[700]	valid_0's multi_logloss: 0.00210355
[800]	valid_0's multi_logloss: 0.00190143
[900]	valid_0's multi_logloss: 0.00175847
[1000]	valid_0's multi_logloss: 0.00167186
[1100]	valid_0's multi_logloss: 0.00159207
[1200]	valid_0's multi_logloss: 0.00155716
[1300]	valid_0's multi_logloss: 0.00152604
[1400]	valid_0's multi_logloss: 0.00150297
[1500]	valid_0's multi_logloss: 0.00149062
[1600]	valid_0's multi_logloss: 0.00147732
[1700]	valid_0's multi_logloss: 0.00147423
[1800]	valid_0's multi_logloss: 0.0014693
[1900]	valid_0's multi_logloss: 0.00144934
[2000]	valid_0's multi_logloss: 0.00144417
[2100]	valid_0's

[2000]	valid_0's multi_logloss: 0.000272167
[2100]	valid_0's multi_logloss: 0.000268609
[2200]	valid_0's multi_logloss: 0.000267459
[2300]	valid_0's multi_logloss: 0.000268256
[2400]	valid_0's multi_logloss: 0.000264957
[2500]	valid_0's multi_logloss: 0.00026451
[2600]	valid_0's multi_logloss: 0.000262567
[2700]	valid_0's multi_logloss: 0.000260302
[2800]	valid_0's multi_logloss: 0.000259361
[2900]	valid_0's multi_logloss: 0.000261875
Early stopping, best iteration is:
[2796]	valid_0's multi_logloss: 0.000258805
[3 6 8 ... 4 5 5]
[0.9996, 0.9998, 1.0, 0.9996, 1.0, 0.9994, 0.9998]
************************************ 8 ************************************
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_logloss: 0.246935
[200]	valid_0's multi_logloss: 0.0460563
[300]	valid_0's multi_logloss: 0.010075
[400]	valid_0's multi_logloss: 0.003051
[500]	valid_0's multi_logloss: 0.00145284
[600]	valid_0's multi_logloss: 0.00086279
[700]	valid_0's multi_logloss:

In [209]:
#catboost
def cv_model(x_train, y_train, x_test, clf_name='cb'):
    folds = 10
    seed = 111
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros((x_train.shape[0],9))
    test = np.zeros((x_test.shape[0],9))


    cv_scores = []


    for i, (train_index, valid_index) in enumerate(kf.split(x_train, y_train)):  
        

        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = x_train.iloc[train_index], y_train[train_index], x_train.iloc[valid_index], y_train[valid_index]
        model =cb.CatBoostClassifier(iterations=3000 ,od_type='Iter',od_wait=120,max_depth=8,learning_rate=0.05,l2_leaf_reg=9,
                             random_seed=2021,metric_period=50,fold_len_multiplier=1.1,loss_function='MultiClass',logging_level='Verbose',
                                     task_type='GPU')


        model = model.fit(trn_x,trn_y)
        print('===')
        val_pred = model.predict_proba(val_x)
        test_pred = model.predict_proba(x_test)
        train[valid_index] = val_pred
        test += test_pred / kf.n_splits
        val_predcol = val_pred.argmax(axis=1)
        cv_scores.append(accuracy_score(val_y, val_predcol)) 
        print('====',cv_scores)
    print("%s_scotrainre_list:" % clf_name , cv_scores)
    print("%s_score_mean:" % clf_name , np.mean(cv_scores))
    print("%s_score_std:" % clf_name , np.std(cv_scores))
    return train, test

In [210]:
cb_train, cb_test = cv_model_cb(x_train, y_train, x_test)

************************************ 1 ************************************


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 1.8728740	total: 23.5ms	remaining: 1m 10s
50:	learn: 0.1465418	total: 1.12s	remaining: 1m 4s
100:	learn: 0.0554430	total: 2.03s	remaining: 58.3s
150:	learn: 0.0374139	total: 2.94s	remaining: 55.4s
200:	learn: 0.0278966	total: 3.89s	remaining: 54.2s
250:	learn: 0.0230376	total: 4.81s	remaining: 52.7s
300:	learn: 0.0194354	total: 5.78s	remaining: 51.8s
350:	learn: 0.0162094	total: 6.72s	remaining: 50.7s
400:	learn: 0.0138033	total: 7.67s	remaining: 49.7s
450:	learn: 0.0115042	total: 8.65s	remaining: 48.9s
500:	learn: 0.0098169	total: 9.57s	remaining: 47.7s
550:	learn: 0.0085534	total: 10.5s	remaining: 46.6s
600:	learn: 0.0075512	total: 11.4s	remaining: 45.7s
650:	learn: 0.0068686	total: 12.4s	remaining: 44.6s
700:	learn: 0.0061886	total: 13.3s	remaining: 43.6s
750:	learn: 0.0056692	total: 14.2s	remaining: 42.6s
800:	learn: 0.0052387	total: 15.1s	remaining: 41.6s
850:	learn: 0.0048088	total: 16s	remaining: 40.5s
900:	learn: 0.0044812	total: 16.9s	remaining: 39.5s
950:	learn: 0.0

1600:	learn: 0.0019832	total: 29s	remaining: 25.4s
1650:	learn: 0.0019018	total: 29.9s	remaining: 24.5s
1700:	learn: 0.0018254	total: 30.8s	remaining: 23.6s
1750:	learn: 0.0017599	total: 31.8s	remaining: 22.6s
1800:	learn: 0.0017053	total: 32.7s	remaining: 21.7s
1850:	learn: 0.0016542	total: 33.6s	remaining: 20.8s
1900:	learn: 0.0016019	total: 34.5s	remaining: 19.9s
1950:	learn: 0.0015426	total: 35.4s	remaining: 19s
2000:	learn: 0.0014940	total: 36.3s	remaining: 18.1s
2050:	learn: 0.0014490	total: 37.2s	remaining: 17.2s
2100:	learn: 0.0014073	total: 38.1s	remaining: 16.3s
2150:	learn: 0.0013692	total: 39s	remaining: 15.4s
2200:	learn: 0.0013275	total: 39.9s	remaining: 14.5s
2250:	learn: 0.0012884	total: 40.8s	remaining: 13.6s
2300:	learn: 0.0012529	total: 41.7s	remaining: 12.7s
2350:	learn: 0.0012216	total: 42.6s	remaining: 11.8s
2400:	learn: 0.0011893	total: 43.5s	remaining: 10.8s
2450:	learn: 0.0011596	total: 44.4s	remaining: 9.94s
2500:	learn: 0.0011285	total: 45.3s	remaining: 9.04s

0:	learn: 1.8725290	total: 23.3ms	remaining: 1m 9s
50:	learn: 0.1470018	total: 1.11s	remaining: 1m 3s
100:	learn: 0.0567595	total: 2.03s	remaining: 58.3s
150:	learn: 0.0367569	total: 2.93s	remaining: 55.3s
200:	learn: 0.0273774	total: 3.82s	remaining: 53.2s
250:	learn: 0.0217781	total: 4.71s	remaining: 51.6s
300:	learn: 0.0174883	total: 5.63s	remaining: 50.5s
350:	learn: 0.0144735	total: 6.55s	remaining: 49.4s
400:	learn: 0.0120144	total: 7.46s	remaining: 48.4s
450:	learn: 0.0098887	total: 8.38s	remaining: 47.4s
500:	learn: 0.0082945	total: 9.3s	remaining: 46.4s
550:	learn: 0.0072298	total: 10.2s	remaining: 45.4s
600:	learn: 0.0063773	total: 11.1s	remaining: 44.4s
650:	learn: 0.0057371	total: 12s	remaining: 43.4s
700:	learn: 0.0051625	total: 12.9s	remaining: 42.4s
750:	learn: 0.0047115	total: 13.8s	remaining: 41.4s
800:	learn: 0.0043495	total: 14.7s	remaining: 40.5s
850:	learn: 0.0040031	total: 15.7s	remaining: 39.5s
900:	learn: 0.0036773	total: 16.6s	remaining: 38.6s
950:	learn: 0.003

1500:	learn: 0.0019900	total: 27.6s	remaining: 27.6s
1550:	learn: 0.0018991	total: 28.6s	remaining: 26.7s
1600:	learn: 0.0018310	total: 29.5s	remaining: 25.8s
1650:	learn: 0.0017514	total: 30.4s	remaining: 24.8s
1700:	learn: 0.0016868	total: 31.3s	remaining: 23.9s
1750:	learn: 0.0016237	total: 32.2s	remaining: 23s
1800:	learn: 0.0015747	total: 33.1s	remaining: 22.1s
1850:	learn: 0.0015228	total: 34.1s	remaining: 21.1s
1900:	learn: 0.0014678	total: 35s	remaining: 20.2s
1950:	learn: 0.0014228	total: 35.9s	remaining: 19.3s
2000:	learn: 0.0013833	total: 36.8s	remaining: 18.4s
2050:	learn: 0.0013386	total: 37.7s	remaining: 17.4s
2100:	learn: 0.0013008	total: 38.6s	remaining: 16.5s
2150:	learn: 0.0012627	total: 39.5s	remaining: 15.6s
2200:	learn: 0.0012316	total: 40.4s	remaining: 14.7s
2250:	learn: 0.0011989	total: 41.3s	remaining: 13.8s
2300:	learn: 0.0011652	total: 42.2s	remaining: 12.8s
2350:	learn: 0.0011356	total: 43.2s	remaining: 11.9s
2400:	learn: 0.0011044	total: 44.1s	remaining: 11s

2950:	learn: 0.0009195	total: 54.8s	remaining: 910ms
2999:	learn: 0.0008994	total: 55.7s	remaining: 0us
===
==== [0.9996, 1.0, 1.0, 0.9996, 0.9998, 0.9986, 0.9992, 0.9996, 0.9996, 0.9992]
cb_scotrainre_list: [0.9996, 1.0, 1.0, 0.9996, 0.9998, 0.9986, 0.9992, 0.9996, 0.9996, 0.9992]
cb_score_mean: 0.9995200000000001
cb_score_std: 0.0004019950248448344


In [ ]:
train_1 = np.zeros((x_train.shape[0],36))
test_1 = np.zeros((x_test.shape[0],36))
train_1[:,0:9] = lgb_train**2
test_1[:,0:9] = lgb_test**2
train_1[:,9:18] = np.exp(lgb_train)
test_1[:,9:18] = np.exp(lgb_test)
train_1[:,18:27] = cb_train**2
test_1[:,18:27] = cb_test**2
train_1[:,27:36] = np.exp(cb_train)
test_1[:,27:36] = np.exp(cb_test)

In [212]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
train_ = pd.DataFrame(train_1)
test_ = pd.DataFrame(test_1)
clf = LogisticRegression()
#clf = GradientBoostingClassifier(learning_rate=0.02, subsample=0.5, max_depth=6, n_estimators=30)
#clf= RandomForestClassifier(n_estimators=50, n_jobs=-1, criterion='gini')
clf.fit(train_, y_train)
y_emb = clf.predict_proba(test_)

In [214]:
y_pre = y_emb.argmax(axis=1)

In [215]:
y_pre

array([2, 5, 1, ..., 1, 6, 5], dtype=int64)

In [216]:
## 预测结果
submit = pd.DataFrame()
submit['id'] = test_data['id']
submit['label'] = y_pre
submit['label'] = submit['label'].apply(lambda x:int(x) if x>0 else 0).values
submit.to_csv('./result.csv',index=False)